In [1]:
import numpy as np
import time
from flatland.envs.rail_generators import complex_rail_generator, rail_from_manual_specifications_generator, random_rail_generator , sparse_rail_generator
from flatland.envs.schedule_generators import complex_schedule_generator, sparse_schedule_generator
from flatland.envs.rail_env import RailEnv
from flatland.utils.rendertools import RenderTool
from flatland.envs.observations import TreeObsForRailEnv,GlobalObsForRailEnv,LocalObsForRailEnv
from flatland.envs.predictions import ShortestPathPredictorForRailEnv
from flatland.utils.ordered_set import OrderedSet
from flatland.core.grid.grid4_utils import get_new_position
# from flatland.utils.observation_utils import normalize_observation
import math

In [2]:
Global_observations = GlobalObsForRailEnv()
Tree_observations = TreeObsForRailEnv(max_depth=2 , predictor=ShortestPathPredictorForRailEnv(max_depth=10) )

In [3]:
def environment1 ():
    NUMBER_OF_AGENTS = 20
    env = RailEnv(
                width=20,
                height=20,
                rail_generator=complex_rail_generator(
                                        nr_start_goal=20,
                                        nr_extra=1,
                                        min_dist=8,
                                        max_dist=99999,
                                        seed=0),
                schedule_generator=complex_schedule_generator(),
                number_of_agents=NUMBER_OF_AGENTS)
    return env


In [4]:
def environment2 ():
    specs = [[(0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0)],
             [(0, 0), (0, 0), (0, 0), (0, 0), (7, 0), (0, 0)],
             [(7, 270), (1, 90), (1, 90), (1, 90), (2, 90), (7, 90)],
             [(0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0)]]

    env = RailEnv(width=6,
                  height=4,
                  rail_generator=rail_from_manual_specifications_generator(specs),
                  number_of_agents=1,
                 observations = Tree_observations
                 )
    
    return env

In [5]:
def random_environment ():
    transition_probability = [1.0,  # empty cell - Case 0
                          1.0,  # Case 1 - straight
                          1.0,  # Case 2 - simple switch
                          0.3,  # Case 3 - diamond drossing
                          0.5,  # Case 4 - single slip
                          0.5,  # Case 5 - double slip
                          0.2,  # Case 6 - symmetrical
                          0.0,  # Case 7 - dead end
                          0.2,  # Case 8 - turn left
                          0.2,  # Case 9 - turn right
                          1.0]  # Case 10 - mirrored switch

    # Example generate a random rail
    env = RailEnv(width=10,
                  height=10,
                  rail_generator=random_rail_generator(
                            cell_type_relative_proportion=transition_probability
                            ),
                  number_of_agents=3,
                  obs_builder_object= Tree_observations )
    return env

In [6]:
def sparse_env ():
    rail_generator = sparse_rail_generator(
                        max_num_cities=20,
                        grid_mode=False,
                        max_rails_between_cities=2,
                        max_rails_in_city=8,
                        seed = 14
#                     num_cities=10,  # Number of cities in map
#                     num_intersections=10,  # Number of interesections in map
#                     num_trainstations=50,  # Number of possible start/targets on map
#                     min_node_dist=6,  # Minimal distance of nodes
#                     node_radius=3,  # Proximity of stations to city center
#                     num_neighb=3,  # Number of connections to other cities
#                     seed=5,  # Random seed
#                     grid_mode=False  # Ordered distribution of nodes
                    )

    #Build the environment
    env = RailEnv(
        width=50,
        height=50,
        rail_generator=rail_generator,
        schedule_generator=sparse_schedule_generator(),
        number_of_agents=3,
        obs_builder_object=TreeObsForRailEnv(max_depth=3)
    )
    return env

In [7]:
env = sparse_env()
env_renderer = RenderTool(env ,screen_width=1600 , screen_height=750 , show_debug=True )
obs , info = env.reset()

/home/arpit/anaconda3/envs/IA/lib/python3.7/site-packages/flatland/envs/rail_generators.py:722: UserWarning: Could not set all required cities!
  "Could not set all required cities!")


In [8]:
def my_controller():
    """
    You are supposed to write this controller
    """
    _action = {}
    for _idx in range(env.get_num_agents()):
        _action[_idx] = np.random.randint(0, 5)
    return _action

In [9]:
_done = False
for _ in range(5):
    _action = my_controller()
    obs, all_rewards, done, _ = env.step(_action)
    _done = done['__all__']
    env_renderer.render_env(show=True , frames = True , show_observations = True , show_predictions=True )
    time.sleep(0.3)

/home/arpit/anaconda3/envs/IA/lib/python3.7/site-packages/flatland/utils/rendertools.py:313: UserWarning: Predictor did not provide any predicted cells to render.                 Predictors builder needs to populate: env.dev_pred_dict
  Predictors builder needs to populate: env.dev_pred_dict")


In [10]:
print(env.get_num_agents())

3


$n \times m$

In [10]:
def print_node (node):
    print("dist_own_target_encountered : " , node.dist_own_target_encountered)
    print("dist_other_target_encountered : ", node.dist_other_target_encountered)
    print("dist_other_agent_encountered : " , node.dist_other_agent_encountered)
    print("dist_potential_conflict : " , node.dist_potential_conflict)
    print("dist_unusable_switch : " , node.dist_unusable_switch)
    print("dist_to_next_branch : " , node.dist_to_next_branch)
    print("dist_min_to_target : " , node.dist_min_to_target)
    print("num_agents_same_direction : " , node.num_agents_same_direction)
    print("num_agents_opposite_direction : " , node.num_agents_opposite_direction )
    print("num_agents_malfunctioning : " , node.num_agents_malfunctioning )
    print("speed_min_fractional : " , node.speed_min_fractional)
    print("num_agents_ready_to_depart " , node.num_agents_ready_to_depart )

In [11]:
# distance_map = env.distance_map.get()[2]
obs[0].childs['L'].childs['F'].childs
# print(distance_map[0,4,0])

{}

# Working on flattening the tree

In [20]:
def max_lt(seq, val):
    """
    Return greatest item in seq for which item < val applies.
    None is returned if seq was empty or all items in seq were >= val.
    """
    max = 0
    idx = len(seq) - 1
    while idx >= 0:
        if seq[idx] < val and seq[idx] >= 0 and seq[idx] > max:
            max = seq[idx]
        idx -= 1
    return max

In [21]:

def min_gt(seq, val):
    """
    Return smallest item in seq for which item >= val applies.
    None is returned if seq was empty or all items in seq were >= val.
    """
    min = np.inf
    idx = len(seq) - 1
    while idx >= 0:
        if seq[idx] >= val and seq[idx] < min:
            min = seq[idx]
        idx -= 1
    return min


In [22]:

def norm_obs_clip(obs, clip_min=-1, clip_max=1, fixed_radius=0, normalize_to_range=False):
    """
    This function returns the difference between min and max value of an observation
    :param obs: Observation that should be normalized
    :param clip_min: min value where observation will be clipped
    :param clip_max: max value where observation will be clipped
    :return: returnes normalized and clipped observatoin
    """
    if fixed_radius > 0:
        max_obs = fixed_radius
    else:
        max_obs = max(1, max_lt(obs, 1000)) + 1

    min_obs = 0  # min(max_obs, min_gt(obs, 0))
    if normalize_to_range:
        min_obs = min_gt(obs, 0)
    if min_obs > max_obs:
        min_obs = max_obs
    if max_obs == min_obs:
        return np.clip(np.array(obs) / max_obs, clip_min, clip_max)
    norm = np.abs(max_obs - min_obs)
    return np.clip((np.array(obs) - min_obs) / norm, clip_min, clip_max)


In [23]:
def _split_node_into_feature_groups(node : TreeObsForRailEnv.Node) -> (np.ndarray , np.ndarray , np.ndarray):
    data = np.zeros(6)
    distance = np.zeros(1)
    agent_data = np.zeros(4)
    
    data[0] = node.dist_own_target_encountered
    data[1] = node.dist_other_target_encountered
    data[2] = node.dist_other_agent_encountered
    data[3] = node.dist_potential_conflict
    data[4] = node.dist_unusable_switch
    data[5] = node.dist_to_next_branch

    distance[0] = node.dist_min_to_target
    
    agent_data[0] = node.num_agents_same_direction
    agent_data[1] = node.num_agents_opposite_direction
    agent_data[2] = node.num_agents_malfunctioning
    agent_data[3] = node.speed_min_fractional


    return data , distance, agent_data


In [24]:
def _split_subtree_into_feature_groups(node: TreeObsForRailEnv.Node, current_tree_depth: int, max_tree_depth: int) -> (np.ndarray, np.ndarray, np.ndarray):
    
    if node == -np.inf:
        remaining_depth = max_tree_depth - current_tree_depth
        num_remaining_nodes = int((4**(remaining_depth+1) - 1) / (4 - 1))
        return [-np.inf] * num_remaining_nodes*6, [-np.inf] * num_remaining_nodes, [-np.inf] * num_remaining_nodes*4


    data, distance, agent_data = _split_node_into_feature_groups(node)

    if not node.childs:
        return data, distance, agent_data
    
    for direction in TreeObsForRailEnv.tree_explored_actions_char:
        sub_data, sub_distance, sub_agent_data = _split_subtree_into_feature_groups(node.childs[direction], current_tree_depth + 1, max_tree_depth)
        data = np.concatenate((data, sub_data))
        distance = np.concatenate((distance, sub_distance))
        agent_data = np.concatenate((agent_data, sub_agent_data))

    
    return data, distance ,agent_data

In [25]:
def split_tree_into_feature_groups(tree : TreeObsForRailEnv.Node , max_tree_depth : int) -> (np.ndarray, np.ndarray , np.ndarray):
    data , distance, agent_data = _split_node_into_feature_groups(tree)

    for direction in TreeObsForRailEnv.tree_explored_actions_char:
        sub_data  , sub_distance, sub_agent_data = _split_subtree_into_feature_groups(tree.childs[direction], 1, max_tree_depth)
        data = np.concatenate((data, sub_data))
        distance = np.concatenate((distance, sub_distance))
        agent_data = np.concatenate((agent_data, sub_agent_data))

    return data , distance , agent_data

In [26]:
def flat_the_tree (observation : TreeObsForRailEnv.Node , tree_depth : int , observation_radius = 0):
    
    data , distance , agent_data = split_tree_into_feature_groups(observation , tree_depth)
    
    data = norm_obs_clip(data , fixed_radius=observation_radius)
    distance = norm_obs_clip(distance, normalize_to_range = True)
    agent_data = np.clip(agent_data , -1 , 1)
    
    normalized_obs = np.concatenate((np.concatenate((data, distance)), agent_data))
    
    return normalized_obs

In [28]:
normalized_obs = flat_the_tree(obs[0] , 2)

In [30]:
normalized_obs.size

231

# Finding the shortest path

In [13]:
#printing the shortest path
def show_path(handle , position = None, direction = None):
    agent = env.agents[handle]
    visited = OrderedSet()
    if (position is None):
        position = agent.position
    if (direction is None):
        direction = agent.direction
    
    distance_map = env.distance_map.get()[handle]
    distance = distance_map[position[0] , position[1] , direction]
    print(position , direction, distance)
    if math.isinf(distance):
        env_renderer.render_env(show=True , frames = False , show_observations = True )
        return 
    
    while not (agent.target == position):
        possible_transitions = env.rail.get_transitions(*position, direction)
        visited.add(position)
        for d in range(4):
            if possible_transitions[d]:
                new_position = get_new_position(position , d)
                if (distance_map[new_position[0] , new_position[1] , d] == distance - 1 ):
                    position = new_position
                    direction = d
                    distance -= 1
                    break

    env.dev_obs_dict[0] = visited
    env_renderer.render_env(show=True , frames = False , show_observations = True )
    

In [23]:
show_path(0 , (3 , 2) , 1)

(3, 2) 1 14.0


In [10]:
action_char = ['L' , 'F' , 'R' , 'B' ]
def take_actions (position , direction):
    trans = env.rail.get_transitions(*position, direction)
    for i, d in enumerate([(direction + i) % 4 for i in range(-1, 3)]):
        print(action_char[i], end = ' ')
        if (trans[d]):
            print(get_new_position(position , d) , d) 
        else:
            print("Not Allowed")

In [11]:
take_actions(position=(3,1) , direction=1 )

L Not Allowed
F Not Allowed
R (4, 1) 2
B Not Allowed


In [14]:
distance_map = env.distance_map.get()[0]
for i in range(4):
    print(distance_map[:,:,i])

[[inf inf  9.  8.  7. inf 11. 12. 13. inf]
 [inf inf  8.  9.  8.  9. inf 13. 14. inf]
 [inf inf  9. inf  9. inf 11. inf inf inf]
 [inf 15. 10. 15. 10. inf 12.  5. inf inf]
 [inf inf inf 16. 11. inf 13.  0. inf inf]
 [inf inf inf 17. 12.  7. inf inf inf inf]
 [inf 18. 19. inf inf  8. 23. 16. inf inf]
 [inf 19. 12. inf 18. inf inf 17. inf inf]
 [inf inf 17. 20. inf inf inf 18. inf inf]
 [inf inf inf inf inf inf inf inf inf inf]]
[[inf inf inf inf inf inf inf inf inf inf]
 [inf inf inf  7. inf  5. 12. 13. 14. inf]
 [inf inf inf  6.  5.  4.  3. 14. 15. 14.]
 [inf 13. 14. 13. 12. 13. 12. inf  4.  3.]
 [inf 16. 11. 16. inf inf inf  0.  3.  2.]
 [inf inf inf 17. inf inf  6.  5.  4.  3.]
 [inf 16. 19. 18.  9.  8. 23. 22. inf inf]
 [inf 13. 12. 11. 10.  9. 18. 17. inf inf]
 [inf 14. 13. inf 19. inf inf inf inf inf]
 [inf inf inf inf inf inf inf inf inf inf]]
[[inf inf inf inf inf inf inf inf inf inf]
 [inf inf  8.  7.  6. inf 10. 11. 12. inf]
 [inf inf  7.  6. 13.  4. inf 12. 13. inf]
 [inf inf